In [1]:
import csv
import pandas as pd
from neo4j import GraphDatabase
from py2neo import Graph

In [7]:
graph = Graph("bolt://34.201.68.240:33498", auth=("neo4j", "hickory-approval-forties"))

### Write embeddings into the graph

In [9]:
with open("data/movies.emb") as movies_emb:
    next(movies_emb)
    reader = csv.reader(movies_emb, delimiter=' ')
    
    params = []
    for row in reader:
        movie_id = int(row[0])
        params.append({
            'id': movie_id,
            'embedding': [float(item) for item in row[1:]]
        })
    graph.run("""
    UNWIND {params} AS param
    MATCH (m:Movie) WHERE id(m) = param.id
    SET m.embedding = param.embedding
    """, {"params": params})

In [8]:
graph.run("""
MATCH (m:Movie) WHERE EXISTS(m.embedding) RETURN count(m)
""").to_data_frame()

,count(m)
0,9125


In [9]:
graph.run("""
MATCH (m:Movie) RETURN count(m)
""").to_data_frame()

,count(m)
0,9125


All movie nodes have now the embedding property (100 dimensional array)

### Extract data (movie, embedding, genres)

In [12]:
genre_ids = graph.run("""
MATCH (g:Genre) RETURN g.name, id(g)
""").to_data_frame()

In [18]:
movie_genres = graph.run("""
MATCH (m:Movie)-[:IN_GENRE]->(genre)
WITH id(m) AS source, m.embedding as embedding, collect(id(genre)) AS target 
RETURN source, embedding, target
""").to_data_frame()

In [19]:
movie_genres

,source,embedding,target
0,0,"[1.1568369, 4.7216134, -4.829661, 2.67412, 4.1...","[1, 2, 3, 4, 6]"
1,5,"[0.3488815, 6.3227067, -3.6201174, -0.45922923...","[1, 3, 6]"
2,7,"[-0.07164516, 4.885974, -5.827049, -0.01449445...","[4, 9]"
3,8,"[0.10417974, -3.2304595, 3.5939155, 1.1410666,...","[9, 10, 4]"
4,11,"[-1.383354, 1.648171, -3.7612596, -1.3233246, ...",[4]
...,...,...,...
9120,9140,"[0.28404692, 0.21873271, -0.3254155, 1.6711218...","[10, 1, 9]"
9121,9141,"[-0.8980954, -1.6762947, -1.2756481, 0.9909565...","[37, 6, 1, 13]"
9122,9142,"[0.6182944, -0.6889125, -0.38154954, 1.0363413...",[49]
9123,9143,"[1.9886293, 1.4495139, 1.4017138, 0.2425547, 1...",[4]


In [36]:
movies_genres_query = """\
MATCH (genre:Genre)
WITH genre ORDER BY genre.name
WITH collect(id(genre)) AS genres
MATCH (m:Movie)-[:IN_GENRE]->(genre)
WITH genres, id(m) AS source, m.embedding AS embedding, collect(id(genre)) AS target
RETURN source, embedding, [g in genres | CASE WHEN g in target THEN 1 ELSE 0 END] AS genres
"""

with driver.session() as session:
    result = session.run(movies_genres_query)
    df = pd.DataFrame([dict(row) for row in result])

In [37]:
df

,source,embedding,genres
0,0,"[1.1568369, 4.7216134, -4.829661, 2.67412, 4.1...","[0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,5,"[0.3488815, 6.3227067, -3.6201174, -0.45922923...","[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,7,"[-0.07164516, 4.885974, -5.827049, -0.01449445...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,8,"[0.10417974, -3.2304595, 3.5939155, 1.1410666,...","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,11,"[-1.383354, 1.648171, -3.7612596, -1.3233246, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
9120,9140,"[0.28404692, 0.21873271, -0.3254155, 1.6711218...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
9121,9141,"[-0.8980954, -1.6762947, -1.2756481, 0.9909565...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
9122,9142,"[0.6182944, -0.6889125, -0.38154954, 1.0363413...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
9123,9143,"[1.9886293, 1.4495139, 1.4017138, 0.2425547, 1...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [39]:
df.to_csv('data/df.csv', index=False)